In [191]:
import cv2
import numpy as np
import math
import imutils
import urllib
from urllib.request import urlopen
#from urllib import urlopen
t=[[]]
blurValue=7
threshold=10
count=0
cou=0
x1=[]
x2=[]
y1=[]
y2=[]
im=[]
l=[]
ex=[]
d={}
thr=[]
prediction=[]
import pickle
clf = pickle.load(open('modelhand5.sav', 'rb'))
camera = cv2.VideoCapture(1)
#url='http://192.168.43.1:8080/shot.jpg'
while True:
    ret, image = camera.read()
#     imgResp=urlopen(url)
#     imgNp=np.array(bytearray(imgResp.read()),dtype=np.uint8)
#     image=cv2.imdecode(imgNp,-1)
    #image = cv2.imread('p3.png')
    #image = imutils.rotate(image,90)
    cv2.imshow('original', image)
    k = cv2.waitKey(1)
    if k == ord('c'):
        camera.release()
        cv2.destroyAllWindows()
        #image = cv2.resize(image, (28,28), interpolation = cv2.INTER_AREA)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (blurValue, blurValue), 0)
        #ret, thresh = cv2.threshold(blur, threshold, 255, cv2.THRESH_BINARY_INV)
        #kernel = np.ones((3,3),np.uint8)
        thresh = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, blockSize=25, C=11)
        #opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
        #thresh = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)
        im2, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for cnt in contours:
            l.append(cv2.contourArea(cnt))
        l.sort(reverse=True)
        for cnt in contours:
            if cv2.contourArea(cnt)<(l[0]/25):
                q,y,w,h = cv2.boundingRect(cnt)
                for i in range(y,y+h):
                    for j in range(q,q+w):
                        thresh[i][j]=0
#         for i in range(thresh.shape[1]):
#             old=count
#             for j in range(thresh.shape[0]):
#                 if thresh[j][i]==255:
#                     count+=1
#             if count>old:
#                 x=min(x,i)
#             if count>0 and old==count:
#                 x2.append(i)
#                 x1.append(x)
#                 x=1000
#                 count=0
#         if old!=count:
#             x2.append(thresh.shape[1])
#             x1.append(x)
#         count=0
#         for z in range(len(x1)):
#             for i in range(thresh.shape[0]):
#                 old=count
#                 for j in range(x1[z],x2[z]):
#                     if thresh[i][j]==255:
#                         count+=1
#                 if count>old:
#                     y=min(y,i)
#                 if count>0 and old==count:
#                     y2.append(i)
#                     y1.append(y)
#                     y=1000
#                     count=0
#             if old!=count:
#                 y2.append(thresh.shape[0])
#                 y1.append(y)
        kernel = np.ones((5,5), np.uint8)
        th = cv2.dilate(thresh, kernel, iterations=1)
        im2, contours, hierarchy = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        #contour.append(contours[0])
        for m in range(len(contours)):
            x,y,w,h = cv2.boundingRect(contours[m])
            if cv2.contourArea(contours[m])>100 and (w>20 or h>20):
                cv2.rectangle(th,(x,y),(x+w,y+h),(255,255,255),2)
                ex.append(x)
                d[x]=m
        ex.sort(reverse=False)
        rows,cols = thresh.shape[:2]
        [vx,vy,x,y] = cv2.fitLine(contours[d[ex[0]]], cv2.DIST_L2,0,0.01,0.01)
        lefty = int((-x*vy/vx) + y)
        righty = int(((cols-x)*vy/vx)+y)
        slope=-vy/vx
        angle1 = math.degrees(math.atan(slope))
        cv2.line(th,(cols-1,righty),(0,lefty),(255,255,255),2)
        rows,cols = thresh.shape[:2]
        [vx,vy,x,y] = cv2.fitLine(contours[d[ex[len(ex)-1]]], cv2.DIST_L2,0,0.01,0.01)
        lefty = int((-x*vy/vx) + y)
        righty = int(((cols-x)*vy/vx)+y)
        slope=-vy/vx
        angle2 = math.degrees(math.atan(slope))
        print(angle1,angle2)
        if angle1*angle2<0:
            angle=max(angle1,angle2)
        else:
            angle=min(angle1,angle2)
        cv2.line(th,(cols-1,righty),(0,lefty),(255,255,255),2)
        if angle<0:
            angle=-(90+angle)
        else:
            angle=90-angle
        for m in ex:
            cnt=contours[d[m]]
            x,y,w,h = cv2.boundingRect(cnt)
            if abs(angle)>10 and abs(angle)<30:
                y-=3
                h+=3
            elif abs(angle)>30:
                y-=5
                h+=5
            imcrop = thresh[max(y,0):min(y+h,thresh.shape[0]), max(0,x):min(x+w,thresh.shape[1])]
            a=45-h
            b=45-w
            a=max(max(a,0),(w-h))
            b=max(max(b,0),(h-w))
            constan = cv2.copyMakeBorder(imcrop,int(a/2),a-int(a/2),int(b/2),b-int(b/2),cv2.BORDER_CONSTANT)
            imag = cv2.resize(constan, (45,45), interpolation = cv2.INTER_AREA)
            imag = imutils.rotate(imag,angle)
            ret, thr = cv2.threshold(imag, 10, 255, cv2.THRESH_BINARY)
            im2, contour, hierarchy = cv2.findContours(thr, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            l=[]
            for cnt in contour:
                l.append(cv2.contourArea(cnt))
            l.sort(reverse=True)
            for cnt in contour:
                cv2.rectangle(imag,(x,y),(x+w,y+h),(255,255,255),2)
                if cv2.contourArea(cnt)<(l[0]):
                    q,y,w,h = cv2.boundingRect(cnt)
                    for i in range(y,y+h):
                        for j in range(q,q+w):
                            thr[i][j]=0
            #thr = cv2.adaptiveThreshold(imag, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, blockSize=5, C=5)
            kernel = np.ones((5,5), np.uint8)
            thr=cv2.dilate(thr, kernel, iterations=1)
            cou+=1
            cv2.imwrite('t'+str(cou)+'.jpg',thr)
            im1, contou, hierarchy = cv2.findContours(thr, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#             ci = imutils.rotate(thr,25)
#             ret, ki = cv2.threshold(ci, 10, 255, cv2.THRESH_BINARY)
#             for l in range(45):
#                 for j in range(45):
#                     t[0].append(int(ki[l][j]/255))
#             predicted = clf.predict(t)
#             if predicted==1:
#                 prediction.append(10)
#                 t=[[]]
#                 continue
#             else:
#                 t=[[]]
#                 for l in range(45):
#                     for j in range(45):
#                         t[0].append(int(thr[l][j]/255))
#                 predicted = clf.predict(t)
#                 prediction.append(predicted[0])
#                 print(predicted)
#                 t=[[]]
            [vx,vy,x,y] = cv2.fitLine(contou[0], cv2.DIST_L2,0,0.01,0.01)
            slope=-vy/vx
            angle3 = math.degrees(math.atan(slope))
            if angle3 > 20 and angle3<65:
                prediction.append(10)
                continue
            for l in range(45):
                for j in range(45):
                    t[0].append(int(thr[l][j]/255))
            predicted = clf.predict(t)
            prediction.append(predicted[0])
            t=[[]]
        print(prediction)
        break
    elif k == 27:  # press ESC to exit
        break
camera.release()
cv2.destroyAllWindows()

/home/youngdumb/ml/lib/python3.5/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.19.1 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/youngdumb/ml/lib/python3.5/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator MLPClassifier from version 0.19.1 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


-59.55858645818837 -12.036161626793227
[6, 13, 4]


In [190]:
thre=cv2.imread('t9.jpg',0)
cv2.imshow('original', thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [192]:
j=[]
k=[]
l=[]
m=[]
d=[]
e=[]
f=[]
g=[]
h=[]
n=[]
i=0
s=''
while i < len(prediction):
    c=0
    while i<len(prediction) and prediction[i]<=9:
        c=c*10+prediction[i]
        i+=1
    if i<len(prediction) and prediction[i]==15:
        c=3.14
        i+=1
    d.append(c)
    s+=str(c)
    if i<len(prediction):
        j.append(prediction[i])
        if prediction[i]==10:
            s+=' / '
        if prediction[i]==11:
            s+=' * '
        if prediction[i]==12:
            s+=' + '
        if prediction[i]==13:
            s+=' - '
        if prediction[i]==14:
            s+=' ^ '
    i+=1
print(d,j)
ans=d[0]
i=0
while i <len(j):
    if j[i]==14:
        p1=i
        #ans=pow(d[i],d[i+1])
        i+=1
        while i<len(j) and j[i]==14:
            #ans=pow(ans,d[i+1])
            i+=1
        ans=d[i]
        p2=i-1
        while p2>=p1:
            print(d[p2]**ans,d[p2],ans)
            ans=d[p2]**ans
            p2-=1
        if len(e)>0:
            e.pop(len(e)-1)
        e.append(ans)
    else:
        if len(e)>0:
            e.append(d[i+1])
        else:
            e.append(d[i])
            e.append(d[i+1])
        k.append(j[i])
        i+=1
print(e,k)
i=0
while i<len(k):
    if k[i]==10:
        p1=i
        #ans=pow(d[i],d[i+1])
        i+=1
        while i<len(k) and k[i]==10:
            #ans=pow(ans,d[i+1])
            i+=1
        ans=e[i]
        p2=i-1
        while p2>=p1:
            ans=e[p2]/ans
            p2-=1
        if len(f)>0:
            f.pop(len(f)-1)
        f.append(ans)
    else:
        if len(f)>0:
            f.append(e[i+1])
        else:
            f.append(e[i])
            f.append(e[i+1])
        l.append(k[i])
        i+=1
print(f,l)
i=0
while i <len(l):
    if l[i]==11:
        ans=f[i]*f[i+1]
        print(ans,"hello",f[i],i,f[i+1])
        i+=1
        while i<len(l) and l[i]==11:
            ans*=f[i+1]
            print(ans,'hell',i)
            i+=1
        if len(g)>0:
            g.pop(len(g)-1)
        g.append(ans)
    else:
        if len(g)>0:
            g.append(f[i+1])
        else:
            g.append(f[i])
            g.append(f[i+1])
        m.append(l[i])
        i+=1
print(g,m)
i=0
while i < len(m):
    if m[i]==12:
        ans=g[i]+g[i+1]
        i+=1
        while i<len(m) and m[i]==12:
            ans+=g[i+1]
            i+=1
        if len(h)>0:
            h.pop(len(h)-1)
        h.append(ans)
    else:
        if len(h)>0:
            h.append(g[i+1])
        else:
            h.append(g[i])
            h.append(g[i+1])
        n.append(m[i])
        i+=1
print(h,n)
i=0
while i < len(n):
    if n[i]==13:
        ans=h[i]-h[i+1]
        i+=1
        while i<len(n) and n[i]==13:
            ans-=h[i+1]
            i+=1
s+=' = '+str(ans)    
print(ans)
font = cv2.FONT_HERSHEY_SIMPLEX
cv2.putText(image,s,(30,450), font, 1,(255,0,0),2,cv2.LINE_AA)

cv2.imshow('original', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

[6, 4] [13]
[6, 4] [13]
[6, 4] [13]
[6, 4] [13]
[6, 4] [13]
2


In [119]:
d = {0:2,3:5,1:2}  # Initialize empty dictionary.
# d[0] = [1.5, 2.0, 1.0, 0.8] # Place key 0 in d, and map this array to it.
# print(d[0])
# d[1] = [0.5, 1.0]
print(d[3])

5


In [149]:
print(angle2)

-25.470507518582824


In [100]:
ret, tu = cv2.threshold(imag, 10, 255, cv2.THRESH_BINARY)
kernel = np.ones((4,4), np.uint8)
thr = cv2.dilate(thr, kernel, iterations=1)

In [71]:
print(l)

44
